In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion
from scipy.stats import norm
from gammapy.data import DataStore
from gammapy.datasets import MapDataset, Datasets
from gammapy.maps import WcsGeom, MapAxis
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.modeling.models import (
    SkyModel,
    PowerLawSpectralModel,
    PointSpatialModel,
    FoVBackgroundModel,
    GaussianSpatialModel,
    Models
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator, ExcessMapEstimator


In [2]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")
pos = SkyCoord.from_name("MSH 15-52")
obs_table_filtered = data_store.obs_table.select_sky_circle(center=pos, radius=5 * u.deg)
obs_ids = obs_table_filtered["OBS_ID"]
observations = data_store.get_observations(obs_ids)
print(len(observations))

20


In [3]:
energy_axis = MapAxis.from_energy_bounds(1, 10.0, 5, unit="TeV",
                                         per_decade=True)

geom = WcsGeom.create(
    skydir=(pos.ra.value, pos.dec.value),
    binsz=0.02,
    width=(5, 6),
    frame="icrs",
    proj="CAR",
    axes=[energy_axis],
)

# Reduced IRFs are defined in true energy (i.e. not measured energy).
energy_axis_true = MapAxis.from_energy_bounds(
    0.1, 20, 20, unit="TeV", name="energy_true", per_decade=True
)

In [4]:
stacked = MapDataset.create(
    geom=geom, energy_axis_true=energy_axis_true, name="msh-stacked"
)
offset_max = 2.3 * u.deg
maker = MapDatasetMaker()
maker_safe_mask = SafeMaskMaker(
    methods=["offset-max", "aeff-max"], offset_max=offset_max
)
datasets = Datasets()

In [5]:
%%time

for obs in observations:
    # First a cutout of the target map is produced
    cutout = stacked.cutout(
        obs.pointing_radec, width=2 * offset_max, name=f"obs-{obs.obs_id}"
    )
    dataset = maker.run(cutout, obs)
    # The data quality cut is applied
    dataset = maker_safe_mask.run(dataset, obs)
    # fit background model
    datasets.append(dataset)
    stacked.stack(dataset)

<timed exec>:4: GammapyDeprecationWarning: Use observation.get_pointing_icrs(time) instead
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarning: Use observation.pointing or observation.get_pointing_{altaz,icrs} instead
  val = self.fget(obj)
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarning: The radec function is deprecated and may be removed in a future version.
  val = self.fget(obj)
<timed exec>:4: GammapyDeprecationWarning: Use observation.get_pointing_icrs(time) instead
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarning: Use observation.pointing or observation.get_pointing_{altaz,icrs} instead
  val = self.fget(obj)
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarni

<timed exec>:4: GammapyDeprecationWarning: Use observation.get_pointing_icrs(time) instead
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarning: Use observation.pointing or observation.get_pointing_{altaz,icrs} instead
  val = self.fget(obj)
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarning: The radec function is deprecated and may be removed in a future version.
  val = self.fget(obj)
<timed exec>:4: GammapyDeprecationWarning: Use observation.get_pointing_icrs(time) instead
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarning: Use observation.pointing or observation.get_pointing_{altaz,icrs} instead
  val = self.fget(obj)
/Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/astropy/utils/decorators.py:777: GammapyDeprecationWarni

CPU times: user 36.4 s, sys: 3.52 s, total: 39.9 s
Wall time: 40.2 s


In [26]:
spatial_model = GaussianSpatialModel(frame="icrs")
spatial_model.lon_0.value = 228
spatial_model.lat_0.value = -59
spatial_model.sigma.value = 0.1

spectral_model = PowerLawSpectralModel(
    index=2.0,
    amplitude=1.0e-12 * u.Unit("1 / (cm2 s TeV)"),
    reference=1 * u.TeV,
)


sky_model = SkyModel(
    spatial_model=spatial_model, spectral_model=spectral_model, name="msh"
)

In [27]:
datasets.models = sky_model

The first calculation takes time

In [28]:
%%time
datasets.stat_sum()

CPU times: user 19.4 s, sys: 902 ms, total: 20.3 s
Wall time: 20.4 s


308505.06181781495

Second one is faster

In [29]:
%%time
datasets.stat_sum()

CPU times: user 90.8 ms, sys: 2.63 ms, total: 93.4 ms
Wall time: 92.4 ms


308505.06181781495

changing parameters again takes time

In [30]:
spatial_model.sigma.value = 0.7

In [31]:
%%time
datasets.stat_sum()

CPU times: user 20.5 s, sys: 696 ms, total: 21.2 s
Wall time: 21.2 s


308515.25583827886

changing spectral parameters, no

In [32]:
spectral_model.amplitude.value = 1e-11

In [33]:
%%time
datasets.stat_sum()

CPU times: user 89.9 ms, sys: 8.25 ms, total: 98.2 ms
Wall time: 98 ms


308514.71906551335

In [36]:
%load_ext line_profiler

In [49]:
from pyinstrument import Profiler
profiler = Profiler()

In [50]:
datasets.models = sky_model

In [51]:
profiler.start()
datasets.stat_sum()
profiler.stop()

In [55]:
profiler.open_in_browser()

'/var/folders/v4/62d6cvbj72qbyw23mq2pwxl00000gn/T/tmpqnjynstx.html'

In [56]:
profiler.print()


  _     ._   __/__   _ _  _  _ _/_   Recorded: 20:24:59  Samples:  6682
 /_//_/// /_\ / //_// / //_'/ //     Duration: 22.569    CPU time: 22.542
/   _/                      v4.4.0

Program: /Users/ASinha/opt/anaconda3/envs/gammapy-dev/lib/python3.8/site-packages/ipykernel_launcher.py -f /Users/ASinha/Library/Jupyter/runtime/kernel-b46473e3-dcf4-4430-aa01-7c0243f22a13.json

22.569 ZMQInteractiveShell.run_code  IPython/core/interactiveshell.py:3342
└─ 22.569 <module>  ../../../../var/folders/v4/62d6cvbj72qbyw23mq2pwxl00000gn/T/ipykernel_87669/2811408099.py:2
   └─ 22.569 Datasets.stat_sum  ../gammapy/gammapy/datasets/core.py:220
      └─ 22.569 MapDataset.stat_sum  ../gammapy/gammapy/datasets/map.py:1084
         └─ 22.510 MapDataset.npred  ../gammapy/gammapy/datasets/map.py:428
            └─ 22.475 MapDataset.npred_signal  ../gammapy/gammapy/datasets/map.py:478
               ├─ 20.243 MapEvaluator.update  ../gammapy/gammapy/datasets/evaluator.py:158
               │  ├─ 19.678 PSFMa